<a href="https://colab.research.google.com/github/Tiagofv/Forecasting-timeseries/blob/main/forecasting_timeseries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Forecasting-timeseries
O projeto compara duas abordagens para solucionar o problema de forecasting timeseries, comparando uma implementação naive e outra utilizando RNN (LSTM).

# Dataset
O dataset utilizado foi o DEOK_hourly.csv, que contém dados de consumo de energia por hora em Ohio, EUA. O dataset contém as seguintes colunas:
- Datetime: timestamp
- DEOK_MW: demanda de energia em megawatts
dataset https://www.kaggle.com/datasets/robikscube/hourly-energy-consumption?select=DOM_hourly.csv

# Implementação ingenua

A abordagem ingênua implementada neste script é um método simples de previsão baseado em médias históricas. Aqui está uma breve explicação:

Agrupamento de Dados: A abordagem agrupa os dados históricos de demanda de energia por dois fatores:

Dia da semana (0-6, representando segunda a domingo)
Hora do dia (0-23)


Cálculo de Médias: Para cada combinação única de dia e hora, calcula a demanda média de energia (DEOK_MW).
Previsão: Quando solicitado a prever a demanda de energia para uma determinada data e hora:

Determina o dia da semana e a hora para essa data e hora.
Em seguida, retorna a demanda média pré-calculada para essa combinação específica de dia e hora.



Por exemplo, se estiver prevendo para terça-feira às 15h, retornaria a demanda histórica média de todas as terças-feiras às 15h nos dados de treinamento.
Este método captura padrões básicos semanais e diários na demanda de energia, assumindo que esses padrões se repetem consistentemente. É considerado "ingênuo" porque não leva em conta outros fatores potencialmente importantes como clima, tendências de longo prazo ou flutuações recentes na demanda. Sua simplicidade o torna uma boa linha de base para comparação com modelos mais sofisticados.

# Implementação RNN (LSTM)

Foi utilizado uma LSTM treinada por 3 épocas com o framework keras.

#  Resultados

MAE RNN: 861,34
MAE Naive: 7441,36

O modelo RNN (Rede Neural Recorrente) supera significativamente a abordagem ingênua (Naive). O erro do RNN é cerca de 11,6% do erro da abordagem ingênua, representando uma redução de 88,4% no erro.

O MAE (Erro Médio Absoluto) menor do RNN indica que suas previsões estão, em média, muito mais próximas dos valores reais.

A grande diferença no desempenho justifica o uso de técnicas mais avançadas de aprendizado de máquina para este problema.
A abordagem RNN parece capturar padrões e relações que a abordagem naive não captura.

In [3]:
!curl -O https://raw.githubusercontent.com/Tiagofv/Forecasting-timeseries/main/dataset/DEOK_hourly.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1522k  100 1522k    0     0  3539k      0 --:--:-- --:--:-- --:--:-- 3548k


In [1]:
import keras
import pandas as pd
import sklearn.model_selection

def normalize_datetime(df: pd.DataFrame):
    # Extract features from the timestamp
    df['hour'] = df['Datetime'].dt.hour
    df['day_of_week'] = df['Datetime'].dt.dayofweek
    df['month'] = df['Datetime'].dt.month
    df['day_of_year'] = df['Datetime'].dt.dayofyear
    df['day_of_month'] = df['Datetime'].dt.day

    # Normalize these features to be in the range [0, 1]
    df['hour'] = df['hour'] / 23
    df['day_of_week'] = df['day_of_week'] / 6
    df['month'] = df['month'] / 11
    df['day_of_year'] = df['day_of_year'] / 365
    df['day_of_month'] = df['day_of_month'] / 31
    df.drop(columns=['Datetime'], inplace=True)
    return df

def normalize_dataset(df: pd.DataFrame, train_mean: float, train_std: float):
    df['DEOK_MW'] = (df['DEOK_MW'] - train_mean) / train_std
    return df
def load_dataset(path: str):
    # Load the dataset
    dataset = pd.read_csv(path, dtype={'DEOK_MW': float, 'Datetime': str})
    dataset['Datetime'] = pd.to_datetime(dataset['Datetime'])
    # split train and test
    train, test = sklearn.model_selection.train_test_split(dataset, test_size=0.2)
    # create validation dataset
    train, val = sklearn.model_selection.train_test_split(train, test_size=0.2)
    mean = train['DEOK_MW'].mean()
    std = train['DEOK_MW'].std()
    train = normalize_dataset(train, mean, std)
    test = normalize_dataset(test, mean, std)
    val = normalize_dataset(val, mean, std)
    return train, test, val

class NaiveImplementation:
    def __init__(self, dataset: pd.DataFrame):
        self.dataset = dataset
        self.group_by_weekday_hour()

    def group_by_weekday_hour(self):
        self.dataset['day_of_week'] = self.dataset['Datetime'].dt.dayofweek
        self.dataset['hour'] = self.dataset['Datetime'].dt.hour
        self.dataset = self.dataset.groupby(['day_of_week', 'hour']).mean()
        return self.dataset

    def predict(self, datetime, ) -> float:
        weekday = datetime.weekday()
        hour = datetime.hour
        return self.dataset.loc[(weekday, hour)]['DEOK_MW']




In [4]:

train, test, val  = load_dataset('DEOK_hourly.csv')
print(f'Train shape: {train.shape}')
print(f'Test shape: {test.shape}')
naive = NaiveImplementation(train)

# obtain MAE from naive implementation on test set
mae_naive = 0
for index, row in test.iterrows():
    prediction = naive.predict(row['Datetime'])
    mae_naive += abs(row['DEOK_MW'] - prediction)

print(f'MAE Naive {mae_naive:.2f}')

train_normalized = normalize_datetime(train)
train = keras.utils.timeseries_dataset_from_array(train_normalized, train['DEOK_MW'], sequence_length=24, sequence_stride=1, batch_size=32)
val_normalized = normalize_datetime(val)
val = keras.utils.timeseries_dataset_from_array(val_normalized, val['DEOK_MW'], sequence_length=24, sequence_stride=1, batch_size=32)
test_normalized = normalize_datetime(test)
test = keras.utils.timeseries_dataset_from_array(test_normalized, test['DEOK_MW'], sequence_length=24, sequence_stride=1, batch_size=32)

# RNN approach
model = keras.Sequential([
    keras.layers.LSTM(32, input_shape=(24, 6)),
    keras.layers.Dense(1)
])


# print(model.summary())
model.compile(loss='mae', optimizer='adam')
history = model.fit(train, epochs=3, validation_data=val)

# predict on test and calc mae
mae = 0
for batch in test:
    inputs, targets = batch
    prediction = model.predict(inputs)
    for i in range(len(targets)):
        mae += abs(targets[i] - prediction[i][0])
print(20 * '-')
print(f'MAE RNN {mae:.2f}')
print(f'MAE Naive {mae_naive:.2f}')
print(20 * '-')


Train shape: (36952, 2)
Test shape: (11548, 2)
MAE Naive 7460.96
Epoch 1/3
1155/1155 [==============================] - 28s 22ms/step - loss: 0.5365 - val_loss: 0.2301
Epoch 2/3
1155/1155 [==============================] - 30s 26ms/step - loss: 0.1471 - val_loss: 0.1025
Epoch 3/3
1/1 [==============================] - 0s 465ms/step
--------------------
MAE RNN 901.78
MAE Naive 7460.96
--------------------
